Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle
import psutil

Carga de datos

In [2]:
clientes = pd.read_csv('cliente_tabla.csv')
productos = pd.read_csv('producto_tabla.csv')
#sample_submission = pd.read_csv('sample_submission.csv')
town_state  = pd.read_csv('town_state.csv')

In [3]:
train = pd.read_csv('train.csv')

Para hacer seguimiento a la memoria

In [4]:
def variables_por_tamaño():
    variables = {k: sys.getsizeof(v) for k, v in globals().items() if not k.startswith('_')}
    
    variables_en_gb = {k: v / 1024**3 for k, v in variables.items()}
    
    tipos_variables = {k: type(v).__name__ for k, v in globals().items() if not k.startswith('_')}
    
    df = pd.DataFrame({'Variable': list(variables_en_gb.keys()),
                       'Tipo': list(tipos_variables.values()),
                       'Tamaño_GB': list(variables_en_gb.values())})
    
    df = df.sort_values('Tamaño_GB', ascending=False).reset_index(drop=True)
    
    # Mostrar la memoria disponible
    memoria_disponible()
    
    return df



def Gb_por_variable(variable):
    print(sys.getsizeof(variable)/(1024**3))
    memoria_disponible()

def memoria_disponible():
    mem = psutil.virtual_memory()
    mem_total_gb = mem.total / (1024 ** 3)  # Memoria total en GB
    mem_disponible_gb = mem.available / (1024 ** 3)  # Memoria disponible en GB

    print("Memoria Total:", mem_total_gb, "GB")
    print("Memoria Disponible:", mem_disponible_gb, "GB")


In [10]:
train.columns

Index(['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID',
       'Producto_ID', 'Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima',
       'Dev_proxima', 'Demanda_uni_equil'],
      dtype='object')

In [6]:
df_ventas_pivot = pd.pivot_table(data=train,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Venta_uni_hoy').fillna(0)

nombre_archivo = 'Pkl/df_ventas_pivot.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(df_ventas_pivot, archivo_pkl)

In [12]:
df_devoluciones_pivot = pd.pivot_table(data=train,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Dev_uni_proxima').fillna(0) 

nombre_archivo = 'Pkl/df_devoluciones_pivot.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(df_devoluciones_pivot, archivo_pkl)

In [131]:
df_ventas_pivot.head(1+3)

Semana                                                 3    4    5    6    7  \
Agencia_ID Canal_ID Ruta_SAK Cliente_ID Producto_ID                            
1110       7        3301     15766      325          0.0  1.0  0.0  0.0  0.0   
                                        328          0.0  1.0  0.0  0.0  0.0   
                                        1212         3.0  4.0  5.0  0.0  0.0   
                                        1216         4.0  2.0  3.0  1.0  2.0   

Semana                                                 8    9  
Agencia_ID Canal_ID Ruta_SAK Cliente_ID Producto_ID            
1110       7        3301     15766      325          0.0  0.0  
                                        328          0.0  0.0  
                                        1212         4.0  1.0  
                                        1216         5.0  0.0

Para Modelar Unds Vendidas

In [408]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def rf_pred(y_true,periodos):
    y_series = pd.Series(y_true)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    forecaster_rf = ForecasterAutoreg(regressor=model,lags=periodos)
    forecaster_rf.fit(y=y_series)
    y_pred = forecaster_rf.predict(steps=periodos)
    return np.array(y_pred)

def rl_pred(y_true, periodos):
    y_series = np.array(y_true)
    model = LinearRegression()
    periodos_pasados = np.arange(1, len(y_true)+1).reshape(-1, 1)
    model.fit(periodos_pasados, y_series)
    periodos_futuros = np.arange(len(y_true) + 1, len(y_true) + periodos + 1).reshape(-1, 1)
    y_pred = model.predict(periodos_futuros)
    return y_pred

def calcular_MSE(y_true,y_pred):
    return np.mean((y_true - y_pred)**2)

def plot(y_true,y_pred,aplica):
    
    if aplica =='S':
        print(y_true)
        plt.plot(np.concatenate((y_true, y_pred)))
        plt.show()
    
def revisar_linealidad(df,id,periodos,graph='N'):
    y_true = df.iloc[id].to_numpy()
    
    y_train = y_true[:y_true.shape[0]-1]
    y_test = y_true[y_true.shape[0]-1]
    
    cof_coerr = np.abs(np.corrcoef(y_true,df_ventas_pivot.columns)[0][1]) #Correlación positiva
    id_name = df_ventas_pivot.index[id]


    if cof_coerr > 0.75: #Solo regresion Lineal
        pred= rl_pred(y_train,periodos)
    if cof_coerr > 0.5: #Combinación regresion Lineal & RandomForest
        pred= ((rl_pred(y_train,periodos) * cof_coerr) + (rf_pred(y_train,periodos) * cof_coerr))
    else: #Solo RandomForest
        pred= rf_pred(y_train,periodos)

    pred = np.clip(pred, a_min=0, a_max=None)

    plot(y_true,pred,aplica=graph)    
    lista_preds = list(pred.astype('int'))
    lista_preds.append(cof_coerr)
    lista_preds.append(calcular_MSE(y_test,pred[0]))
    return id_name,lista_preds

revisar_linealidad(df_ventas_pivot,8,3,graph='N')

((1110, 7, 3301, 15766, 1250),
 [10, 10, 11, 0.5439433649162874, 0.0081269177815803])

In [ ]:
lista_proyecciones = []
for num,codigo in enumerate(df_ventas_pivot.index):
    print(num/df_ventas_pivot.shape[0])
    lista_proyecciones.append(revisar_linealidad(df_ventas_pivot,num,3))

Base de datos test

In [413]:
test  = pd.read_csv('test.csv')
test_pivot = pd.pivot_table(data=test,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Semana')

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277
...,...,...,...,...,...,...,...
6999246,6999246,11,2057,1,1153,4379638,1232
6999247,6999247,10,1334,1,2008,970421,43069
6999248,6999248,11,1622,1,2869,192749,30532
6999249,6999249,11,1636,1,4401,286071,35107


Indices faltantes total

In [681]:
indices_train = df_ventas_pivot.index
indices_test = test_pivot.index

indices_faltantes_total = indices_test.difference(indices_train)
indices_faltantes_total.shape

(1355454,)

In [453]:
Diccionario_faltantes={}
def verificar_agencias(col):
    indices_izq = test_pivot.reset_index()[col].unique()
    indices_der = df_ventas_pivot.reset_index()[col].unique()

    faltantes= np.setdiff1d(indices_izq, indices_der)

    Diccionario_faltantes[col]=faltantes
    print(col,str(faltantes))


for col in ['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID']:
    verificar_agencias(col)

#np.setdiff1d(Diccionario_faltantes['Producto_ID'],productos['Producto_ID'].unique())

Agencia_ID []
Canal_ID []
Ruta_SAK [1823 3555 3556 3557 3558 3801 3803 4559 6148 6162 7433 7775 8044 8189
 9349 9562 9585]
Cliente_ID [   2683    5380    5707 ... 9746104 9747048 9780515]
Producto_ID [   98 31203 31211 31655 32026 32224 32421 32591 32798 32820 33053 35191
 35246 36524 36673 37202 37362 37404 37405 37494 37495 37496 37610 37617
 37618 37620 37626 37688 37702 37745 42323 46064 46131 48217]


In [700]:
columnas_per_level={'Agencia_ID':0,'Canal_ID':1,'Ruta_SAK':2,'Cliente_ID':3,'Producto_ID':4}

def buscar_un_registro(col,prod_id):
    indx = test_pivot.xs(prod_id,  level=columnas_per_level[col], drop_level=False).index    
    lista_ixs=[]
    for ix in indx:
        lista_ixs.append({indices_test.get_loc(ix):ix})
    return lista_ixs

buscar_un_registro('Producto_ID',48217)

[{6113671: (22560, 1, 1077, 801976, 48217)},
 {6113698: (22560, 1, 1077, 1024404, 48217)}]

Nombres similares

Ajuste de nombres de productos

In [546]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity


def limpiar_texto(texto):
    texto_limpio = re.sub(r'\b\w{1,2}\b|\d+', '', texto.lower())
    return texto_limpio

productos['NombreProductoLimpio'] = productos['NombreProducto'].apply(limpiar_texto)
productos['NombreProductoLimpio'] = productos['NombreProductoLimpio'].apply(limpiar_texto)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(productos['NombreProductoLimpio'])
count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(),index=productos['Producto_ID'])
similaridades = cosine_similarity(count_df)

Buscar ids

In [608]:
def Productos_similares(id_a_buscar,graph='N'):
    similares = similaridades[productos[productos['Producto_ID'] == id_a_buscar].index]
    similares_df = pd.DataFrame(similares.T, index=productos['Producto_ID'], columns=['Similitud'])
    productos_similares = similares_df[similares_df.index != id_a_buscar].sort_values(by='Similitud', ascending=False).head(5)

    if graph=='S':
        display(productos[productos['Producto_ID']==id_a_buscar])
        display(productos[productos['Producto_ID'].isin(productos_similares.index)])
    return productos[productos['Producto_ID'].isin(productos_similares.head(5).index)]['Producto_ID'].to_numpy()

Productos_similares(31655,graph='N')

array([32851, 35121, 35123, 35124, 35125], dtype=int64)

In [732]:
archivo= 'Pkl/Cluster_info.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    Cluster_info = pickle.load(archivo_pkl)

In [742]:
def buscar_productos_por_cluster(tupla):
    Cluster_info[0]

In [731]:
def Predict_test(df,num):
    
    Dict_nombres = {}
    index_de_test = df.iloc[num].name

    Dict_nombres['Agencia_ID']=index_de_test[0]
    Dict_nombres['Canal_ID']=index_de_test[1]
    Dict_nombres['Ruta_SAK']=index_de_test[2]
    Dict_nombres['Cliente_ID']=index_de_test[3]
    Dict_nombres['Producto_ID']=index_de_test[4]

    if index_de_test in indices_faltantes_total:
        #Si no existe la tupla verificamos si existe el id producto en train
        if Dict_nombres['Producto_ID'] in Diccionario_faltantes['Producto_ID']:
            #Si no existe el id producto buscamos el similar por distancia de coseno
            ids_similares = Productos_similares(Dict_nombres['Producto_ID'],graph='N')
            #Revisamos cada uno de los resultados
            lista_resultados=i
            for ids_sim in ids_similares:
                #Creamos la nueva tupla con los ids similares
                tuple_id_sim = (Dict_nombres['Agencia_ID'],Dict_nombres['Canal_ID'],Dict_nombres['Ruta_SAK'],Dict_nombres['Cliente_ID'],ids_sim)
                if tuple_id_sim in indices_train:
    
                    revisar_linealidad(df_ventas_pivot,indices_train.get_loc(tuple_id_sim),3,graph='N')
                    
    else:
        return revisar_linealidad(df_ventas_pivot,indices_train.get_loc(index_de_test),3,graph='N')
    
Predict_test(test_pivot,6113671)

In [728]:
df_ventas_pivot.loc[(22560,1,1077,801976)]

Semana,3,4,5,6,7,8,9
Producto_ID,,,,,,,
693,0.0,0.0,0.0,1.0,0.0,0.0,0.0
815,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1109,0.0,2.0,5.0,0.0,0.0,1.0,2.0
1146,0.0,0.0,4.0,1.0,6.0,4.0,3.0
1150,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1169,0.0,0.0,0.0,0.0,3.0,1.0,1.0
1180,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2233,11.0,6.0,12.0,2.0,6.0,8.0,5.0
3631,0.0,0.0,3.0,0.0,0.0,1.0,1.0


In [721]:
(22560, 1, 1077, 801976, 1876) in indices_faltantes_total

False

In [719]:
indices_faltantes_total

MultiIndex([( 1110, 7, 3301,   50395,   713),
            ( 1110, 7, 3301,   73589,  1644),
            ( 1110, 7, 3301,  181232, 47336),
            ( 1110, 7, 3301,  198780, 42869),
            ( 1110, 7, 3301,  319641,   325),
            ( 1110, 7, 3301,  319641,  1212),
            ( 1110, 7, 3301,  319641,  1643),
            ( 1110, 7, 3301,  319641, 31688),
            ( 1110, 7, 3301,  319641, 32936),
            ( 1110, 7, 3301,  319641, 32961),
            ...
            (25759, 1, 5517, 4483380, 30694),
            (25759, 1, 5517, 4483380, 31005),
            (25759, 1, 5517, 4483380, 34255),
            (25759, 1, 5517, 4483380, 37024),
            (25759, 1, 5517, 4528866, 35142),
            (25759, 1, 5517, 4565941, 36164),
            (25759, 1, 5517, 4565941, 37581),
            (25759, 1, 5517, 4707699, 34226),
            (25759, 1, 5517, 4707699, 36598),
            (25759, 1, 5517, 4744611, 34255)],
           names=['Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Client